# Training Model

In [22]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torch.nn.functional as F

In [23]:
class EmotionRecognitionModel(nn.Module):
    def __init__(self):
        super(EmotionRecognitionModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(64*1*1, 1024)
        self.fc2 = nn.Linear(1024, 7)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.avg_pool2d(x, x.shape[2:])
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

model = EmotionRecognitionModel()

In [24]:
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

trainset = datasets.ImageFolder(root='dataset/kaggle', transform=transform)
trainloader = DataLoader(trainset, batch_size=32, shuffle=True)

testset = datasets.ImageFolder(root='dataset/kaggle', transform=transform)
testloader = DataLoader(testset, batch_size=32, shuffle=True)

In [25]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [ ]:
for i, data in enumerate(trainloader, 0):
    inputs, labels = data
    if labels.max() >= 7:
        print(f'Error in batch {i}, max label = {labels.max()}')

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

loss_values = []

for epoch in range(10):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    loss_values.append(running_loss / len(trainloader))
    print('Epoch %d loss: %.3f' % (epoch + 1, loss_values[-1]))

print('Finished Training')

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the test images: %d %%' % (100 * correct / total))

In [ ]:
torch.save(model.state_dict(), 'model/pytorch/emotion_recognition_model.pth')

In [ ]:
def predict_image(image_path):
    transform = transforms.Compose([
        transforms.Resize((32, 32)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

    model = EmotionRecognitionModel()
    model.load_state_dict(torch.load('model/pytorch/emotion_recognition_model.pth'))
    model.eval()
    
    image = Image.open(image_path)
    image = transform(image).unsqueeze(0)
    with torch.no_grad():
        output = model(image)
        predicted = torch.argmax(output).item()
        
    return predicted  # returns the predicted class

# Test the function
print(predict_image('dataset/kaggle/happiness/2Q__ (10)_face.png'))